In [ ]:
%load_ext autoreload
%autoreload 2

In [47]:
import matplotlib.pyplot as plt

import torch

import pyciemss
from pyciemss.PetriNetODE.interfaces import (setup_petri_model, reset_model, intervene_petri_model,
                                             sample_petri, calibrate_petri, optimize, load_petri_model)
import mira
from pyciemss.utils import get_tspan
from pyciemss.utils.interface_utils import convert_to_output_format
from pyciemss.visuals import vega
import pandas as pd

import warnings
warnings.filterwarnings('ignore')


# Deterministic Lotka Volterra

## Load model

In [3]:
from pyciemss.PetriNetODE.models import LotkaVolterra

raw_deterministic_lotka_volterra = LotkaVolterra(alpha=.67, beta=1.33, gamma=1.00, delta=1.00, add_uncertainty=False)
raw_deterministic_lotka_volterra

LotkaVolterra()

## Initialize model

In [4]:
deterministic_lotka_volterra = setup_petri_model(raw_deterministic_lotka_volterra, start_time=0.0, start_state=dict(prey_population=1.0, predator_population=1.0))
deterministic_lotka_volterra

LotkaVolterra()

In [5]:
tspan = get_tspan(1, 50, 500)

## Sample trajectory

In [6]:
deterministic_lotka_volterra_trajectory = sample_petri(deterministic_lotka_volterra, tspan, 1)
deterministic_lotka_volterra_trajectory['prey_population_sol'][0][0]

tensor(0.5775)

In [13]:
deterministic_lotka_volterra_trajectory_df = convert_to_output_format(deterministic_lotka_volterra_trajectory, tspan)
schema = vega.trajectories(deterministic_lotka_volterra_trajectory_df)
vega.ipy_display(schema)


# Load lotka volterra  with uncertainty

In [14]:
from pyciemss.PetriNetODE.models import LotkaVolterra

uncertain_lotka_volterra_raw= LotkaVolterra(alpha=0.67, beta=1.33, gamma=1.0, delta=1.0, pseudocount=10)
uncertain_lotka_volterra_raw

LotkaVolterra()

## Initialize uncertain Lotka-volterra model

In [15]:
uncertain_lotka_volterra = setup_petri_model(uncertain_lotka_volterra_raw,
                                             start_time=0.0, 
                                             start_state=dict(
                                                 prey_population=1.00, 
                                                 predator_population=1.00))
uncertain_lotka_volterra

LotkaVolterra()

## Sample from prior

In [16]:

prior_samples = sample_petri(uncertain_lotka_volterra, tspan, 50)
prior_samples.keys()

dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [22]:
prior_samples_df = convert_to_output_format(prior_samples, tspan)
schema = vega.trajectories(prior_samples_df, 
                           subset=".*_sol",
                           relabel={"prey_population_sol": "prey_prior",
                                    "predator_population_sol": "predator_prior"})
vega.ipy_display(schema)

## Calibrate uncertain Lotka Volterra

We make one observation per day. Because we simulate 500 data points over 50 days, we need to sample every 10th timepoint.

In [23]:
fitted_parameters = calibrate_petri(uncertain_lotka_volterra, 
                                data =[(i, dict(prey_population=deterministic_lotka_volterra_trajectory['prey_population_sol'][0][i*10]))
                                       for i in range(1,50)])
fitted_parameters

AutoLowRankMultivariateNormal()

In [25]:
posterior_samples = sample_petri(uncertain_lotka_volterra, 
                                 timepoints=tspan, num_samples=50, 
                           inferred_parameters=fitted_parameters)
posterior_samples_df = convert_to_output_format(posterior_samples, tspan)

posterior_samples.keys()


dict_keys(['alpha', 'beta', 'gamma', 'delta', 'prey_population_sol', 'predator_population_sol'])

In [70]:
distributions = pd.DataFrame({
    'timepoint_id': prior_samples_df['timepoint_id'],
    'sample_id': prior_samples_df['sample_id'],
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_posterior': posterior_samples_df['prey_population_sol'],
    'predator_posterior': posterior_samples_df['predator_population_sol'],
    })

traces = pd.DataFrame({
    "timepoint_id": deterministic_lotka_volterra_trajectory_df['timepoint_id'],
    'prey_data': deterministic_lotka_volterra_trajectory_df['prey_population_sol'],
    'predator_data': deterministic_lotka_volterra_trajectory_df['predator_population_sol']
})

schema = vega.trajectories(distributions, traces=traces)
vega.ipy_display(schema)

## Intervene on uncertain Lotka Volterra

In [71]:
intervened_lotka_volterra = intervene_petri_model(uncertain_lotka_volterra,
                                                  [(25, 'alpha', 1.0)])
intervened_lotka_volterra

LotkaVolterra()

## Sample from intervened model

At $t=25$ we intervene to set $\alpha=1.0$.

In [72]:
intervened_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50
                                  )
intervened_samples

{'alpha': tensor([0.6963, 0.6741, 0.7146, 0.6496, 0.7251, 0.6714, 0.7189, 0.6064, 0.7110,
         0.6101, 0.6341, 0.6520, 0.6976, 0.6480, 0.6365, 0.6422, 0.6668, 0.6601,
         0.6518, 0.6156, 0.6690, 0.6106, 0.7181, 0.7223, 0.6698, 0.6433, 0.6455,
         0.6276, 0.7027, 0.6883, 0.6977, 0.6342, 0.6353, 0.6454, 0.6275, 0.6878,
         0.6655, 0.6162, 0.6544, 0.6382, 0.6814, 0.6331, 0.6613, 0.6699, 0.7062,
         0.6617, 0.6570, 0.6053, 0.6789, 0.6803]),
 'beta': tensor([1.2082, 1.3590, 1.4358, 1.3880, 1.3438, 1.2072, 1.2358, 1.2729, 1.4048,
         1.2206, 1.2176, 1.2956, 1.2099, 1.3643, 1.3135, 1.3036, 1.3032, 1.2822,
         1.3061, 1.2204, 1.2064, 1.3107, 1.2007, 1.2226, 1.2870, 1.3390, 1.3497,
         1.2562, 1.3394, 1.2238, 1.2932, 1.2080, 1.2626, 1.3790, 1.2431, 1.2479,
         1.2243, 1.4118, 1.2636, 1.2082, 1.3765, 1.2162, 1.2130, 1.2082, 1.2697,
         1.3570, 1.4304, 1.3373, 1.3633, 1.3446]),
 'gamma': tensor([0.9775, 1.0921, 0.9143, 0.9300, 0.9859, 0.9766, 1.045

In [85]:
intervened_samples_df = convert_to_output_format(intervened_samples, tspan)
distributions = pd.DataFrame({
    'timepoint_id': prior_samples_df['timepoint_id'],
    'sample_id': prior_samples_df['sample_id'],
    'prey_prior': prior_samples_df['prey_population_sol'], 
    'predator_prior': prior_samples_df['predator_population_sol'],
    'prey_intervened': intervened_samples_df['prey_population_sol'],
    'predator_intervened': intervened_samples_df['predator_population_sol'],
    })

schema = vega.trajectories(distributions,
                           markers={"Intervention at T=250": 250})
schema = vega.resize(schema, w=700, h=400)
vega.ipy_display(schema)

## Sample from counterfactual distribution

Given that we observe the deterministic trajectory in the factual world,
what would be the trajectory in a counterfactual world where we intervene at $t=25$ to set $\alpha=1.5$?

In [91]:
counterfactual_samples = sample_petri(intervened_lotka_volterra,
                                  timepoints=tspan, num_samples = 50,
                                      inferred_parameters=fitted_parameters
                                  )
counterfactual_samples

{'alpha': tensor([0.6495, 0.6624, 0.6618, 0.6662, 0.6554, 0.6604, 0.6650, 0.6651, 0.6604,
         0.6528, 0.6589, 0.6573, 0.6609, 0.6610, 0.6613, 0.6563, 0.6641, 0.6617,
         0.6605, 0.6648, 0.6659, 0.6687, 0.6628, 0.6604, 0.6549, 0.6660, 0.6594,
         0.6646, 0.6575, 0.6578, 0.6646, 0.6649, 0.6657, 0.6608, 0.6615, 0.6657,
         0.6592, 0.6659, 0.6572, 0.6576, 0.6594, 0.6560, 0.6546, 0.6638, 0.6663,
         0.6555, 0.6645, 0.6643, 0.6631, 0.6634]),
 'beta': tensor([1.2553, 1.2611, 1.2608, 1.2375, 1.2221, 1.2328, 1.2425, 1.2623, 1.2430,
         1.2327, 1.2402, 1.2466, 1.2180, 1.2211, 1.2336, 1.2272, 1.2674, 1.2700,
         1.2495, 1.2754, 1.2409, 1.2643, 1.2477, 1.2752, 1.2239, 1.2876, 1.2426,
         1.2302, 1.2418, 1.2349, 1.2474, 1.2254, 1.2590, 1.2523, 1.2428, 1.2464,
         1.2382, 1.2424, 1.2435, 1.2388, 1.2672, 1.2538, 1.2275, 1.2760, 1.2604,
         1.2219, 1.2467, 1.2388, 1.2509, 1.2618]),
 'gamma': tensor([1.0315, 1.0168, 1.0213, 1.0070, 1.0215, 1.0189, 1.010

In [94]:
counterfactual_samples_df = convert_to_output_format(counterfactual_samples, tspan)
dists = pd.DataFrame(
    {   'timepoint_id': posterior_samples_df['timepoint_id'],
    'sample_id': posterior_samples_df['sample_id'],
        'prey_posterior': posterior_samples_df['prey_population_sol'], 
     'predator_posterior': posterior_samples_df['predator_population_sol'],
     'prey_counterfactual': counterfactual_samples_df['prey_population_sol'],
     'predator_counterfactual': counterfactual_samples_df['predator_population_sol']
    })

schema = vega.trajectories(dists, markers={"Intervention at T=250": 250})
vega.ipy_display(schema)